In [42]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/usr/bin/python
3.8.14 (default, Sep  8 2022, 00:02:10) 
[GCC 7.5.0]
sys.version_info(major=3, minor=8, micro=14, releaselevel='final', serial=0)


In [43]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.

"""Notebook used for debugging purpose to train the
the DQL agent and then run it one step at a time.
"""

# pylint: disable=invalid-namew

'Notebook used for debugging purpose to train the\nthe DQL agent and then run it one step at a time.\n'

In [44]:
%load_ext autoreload
%autoreload 2

import sys
import logging
import gym
import numpy as np
from ipywidgets import interact
import pprint

import torch
torch.cuda.set_device('cuda:4')

import cyberbattle.agents.baseline.learner as learner
import cyberbattle.agents.baseline.agent_wrapper as w
import cyberbattle.agents.baseline.agent_dql as dqla
from cyberbattle.agents.baseline.agent_wrapper import ActionTrackingStateAugmentation, AgentWrapper, Verbosity
logging.basicConfig(stream=sys.stdout, level=logging.ERROR, format="%(levelname)s: %(message)s")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
gymid = 'CyberBattleToyCtf-v0'
iteration_count = 25
training_episode_count = 50
epsilon_exponential_decay = 50

In [46]:
# Load the gym environment

ctf_env = gym.make(gymid)

ep = ctf_env.env.bounds#w.EnvironmentBounds.of_identifiers(
    #maximum_node_count=5,
    #maximum_total_credentials=10,
    #identifiers=ctf_env.identifiers
#)

current_o = ctf_env.reset()
wrapped_env = AgentWrapper(ctf_env, ActionTrackingStateAugmentation(ep, current_o))
ep = wrapped_env.env.bounds

current_o= ctf_env.reset()['action_mask']['connect'].shape
current_o1= wrapped_env.reset()['action_mask']['connect'].shape
print(current_o, current_o1)

(10, 10, 7, 1000) (10, 10, 7, 1000)


In [47]:
print(wrapped_env.bounds,'\n\n' ,ctf_env.bounds, '\n')
print(ctf_env.env.action_space, '\n\n',ctf_env.sample_valid_action())

EnvironmentBounds(maximum_total_credentials=1000, maximum_node_count=10, maximum_discoverable_credentials_per_action=5, port_count=7, property_count=10, local_attacks_count=3, remote_attacks_count=8) 

 EnvironmentBounds(maximum_total_credentials=1000, maximum_node_count=10, maximum_discoverable_credentials_per_action=5, port_count=7, property_count=10, local_attacks_count=3, remote_attacks_count=8) 

DiscriminatedUnion(connect:MultiDiscrete([  10   10    7 1000]), local_vulnerability:MultiDiscrete([10  3]), remote_vulnerability:MultiDiscrete([10 10  8])) 

 {'remote_vulnerability': array([0, 0, 3], dtype=int32)}


In [16]:
# Evaluate the Deep Q-learning agent
dqn_learning_run = learner.epsilon_greedy_search(
    cyberbattle_gym_env=ctf_env,
    environment_properties=ep,
    learner=dqla.DeepQLearnerPolicy(
        ep=ep,
        gamma=0.015,
        replay_memory_size=10000,
        target_update=5,
        batch_size=512,
        learning_rate=0.01  # torch default learning rate is 1e-2
    ),
    episode_count=training_episode_count,
    iteration_count=iteration_count,
    epsilon=0.90,
    epsilon_exponential_decay=5000,
    epsilon_minimum=0.10,
    verbosity=Verbosity.Quiet,
    render=False,
    plot_episodes_length=False,
    title="DQL"
)

###### DQL
Learning with: episode_count=50,iteration_count=25,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=5
  ## Episode: 1/50 'DQL' ϵ=0.9000, γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=5


Episode 1|Iteration 25|reward:   22.0|last_reward_at:    2|Elapsed Time: 0:00:00||


  Episode 1 stopped at t=25 
  Breakdown [Reward/NoReward (Success rate)]
    explore-local: 1/4 (0.20)
    explore-remote: 1/18 (0.05)
    explore-connect: 0/0 (NaN)
    exploit-local: 0/0 (NaN)
    exploit-remote: 0/1 (0.00)
    exploit-connect: 0/0 (NaN)
  exploit deflected to exploration: 0
  ## Episode: 2/50 'DQL' ϵ=0.8962, γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=5


Episode 2|Iteration 25|reward:   44.0|last_reward_at:   13|Elapsed Time: 0:00:00||


  Episode 2 stopped at t=25 
  Breakdown [Reward/NoReward (Success rate)]
    explore-local: 1/4 (0.20)
    explore-remote: 3/17 (0.15)
    explore-connect: 0/0 (NaN)
    exploit-local: 0/0 (NaN)
    exploit-remote: 0/0 (NaN)
    exploit-connect: 0/0 (NaN)
  exploit deflected to exploration: 6
  ## Episode: 3/50 'DQL' ϵ=0.8922, γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=5


...

  Episode 48 stopped at t=25 
  Breakdown [Reward/NoReward (Success rate)]
    explore-local: 1/4 (0.20)
    explore-remote: 0/11 (0.00)
    explore-connect: 0/0 (NaN)
    exploit-local: 0/0 (NaN)
    exploit-remote: 3/6 (0.33)
    exploit-connect: 0/0 (NaN)
  exploit deflected to exploration: 0
  ## Episode: 49/50 'DQL' ϵ=0.7294, γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=5


Episode 49|Iteration 25|reward:   33.0|last_reward_at:   22|Elapsed Time: 0:00:00||


  Episode 49 stopped at t=25 
  Breakdown [Reward/NoReward (Success rate)]
    explore-local: 1/4 (0.20)
    explore-remote: 2/17 (0.11)
    explore-connect: 0/0 (NaN)
    exploit-local: 0/0 (NaN)
    exploit-remote: 0/1 (0.00)
    exploit-connect: 0/0 (NaN)
  exploit deflected to exploration: 1
  ## Episode: 50/50 'DQL' ϵ=0.7263, γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=5


Episode 50|Iteration 25|reward:   44.0|last_reward_at:   12|Elapsed Time: 0:00:00||


  Episode 50 stopped at t=25 
  Breakdown [Reward/NoReward (Success rate)]
    explore-local: 0/2 (0.00)
    explore-remote: 1/16 (0.06)
    explore-connect: 0/0 (NaN)
    exploit-local: 1/0 (1.00)
    exploit-remote: 2/3 (0.40)
    exploit-connect: 0/0 (NaN)
  exploit deflected to exploration: 3
simulation ended


### Display attack path and learned agent knowledge == env.state as observation

In [17]:
current_o_keys = [
        'newly_discovered_nodes_count', 
        'leaked_credentials', 
        'lateral_move', 
        'customer_data_found', 
        'escalation', 
        #'action_mask', 
        'probe_result', 
        'credential_cache_matrix', 
        'credential_cache_length', 
        'discovered_node_count', 
        'discovered_nodes_properties', 
        'nodes_privilegelevel', 
        '_credential_cache', 
        '_discovered_nodes', 
        #'_explored_network'
                 ]

wrapped_env.action_space

DiscriminatedUnion(connect:MultiDiscrete([  10   10    7 1000]), local_vulnerability:MultiDiscrete([10  3]), remote_vulnerability:MultiDiscrete([10 10  8]))

In [92]:
print(gym_action)
for output in outputs_on_step[1:]:
    for ind in np.where(output['action_mask']):
        print(l.stateaction_model.action_space.abstract_from_gymaction(
            gym_action))

{'connect': array([0, 1, 2, 0], dtype=int32)}
5
5
5
5
5
5


In [20]:
# initialize the environment

current_o = ctf_env.reset()
wrapped_env = AgentWrapper(ctf_env, ActionTrackingStateAugmentation(ep, current_o))
l = dqn_learning_run['learner']

current_o = wrapped_env.reset()
current_initial = current_o.copy()

# Use the trained agent to run the steps one by one

max_steps = 10

# next action suggested by DQL agent
net_bitmap_l = []
outputs_on_step= []
total_reward = 0
for i in range(max_steps):
    sampled_actions = {}
    print("\nStep: {}, sample all actions under action_mask"
          "(bruteforce search)".format(i))
            
    #bruteforce sample actions
    all_local_flag = all_remote_flag = 0
    while True:
        sampled_action =  wrapped_env.sample_valid_action()

        if not all_local_flag:
            if "local" not in str(sampled_action):
                continue
        else:
            if not all_remote_flag and "remote" not in str(sampled_action):
                continue
        
        if str(sampled_action) not in sampled_actions.keys():
            sampled_actions[str(sampled_action)] = 0
            print("\t{}\t{}".format(str(sampled_action),
                                       wrapped_env.pretty_print_internal_action(
                                           sampled_action)))
        else:
            if np.all(np.array(list(sampled_actions.values())) > 5):
                if not all_local_flag:
                    all_local_flag = 1
                    continue
                if not all_remote_flag:
                    all_remote_flag = 1
                    continue
                break
            sampled_actions[str(sampled_action)] += 1

    # run the suggested action
    action_style, gym_action, action_metadata = l.exploit(wrapped_env, current_o)
    if gym_action is None:
        print("Exploit unsuccessful: ", action_style, ' ', gym_action)
        break
    net_bitmap_l.append(
            (wrapped_env.get_explored_network_node_properties_bitmap_as_numpy(current_o), 
             gym_action))

    current_o, reward, done, info = wrapped_env.step(gym_action)
    outputs_on_step += [current_o]  
    
    total_reward += reward
    if True:
        sign = ['-', '+'][reward > 0]
        sign = [sign, '='][-1e-4 < reward < 1e-4]

        print(f"\nExecuted: \n {sign} t={i} {action_style} r={reward} cum_reward:{total_reward} "
              f"a={action_metadata}-{gym_action} "
              f"creds={len(current_o['credential_cache_matrix'])} "
              f" {l.stateaction_as_string(action_metadata)}")
        wrapped_env.render()
    
    if total_reward > 1000:
        print("Successfull flag acquiring")
        break

#     if True and (i == 0 or i >= max_steps - 3):
#         pprint.pprint(current_o)
#         print()

@interact(step=(-1, len(outputs_on_step)-1))
def double_number(step: int):
    if step < 0:
        print("\n\nInitial observation only t=",step)
        pprint.pprint({k: current_initial[k] for k in current_o_keys})
        return
    pprint.pprint("explored network node " + repr(net_bitmap_l[step][0]) 
                  + "\n gym_action" + repr(net_bitmap_l[step][1]))
    print("\n Observation after action t=",step)
    pprint.pprint({k: outputs_on_step[step][k] for k in current_o_keys})
    print("\nAction mask ")
    pprint.pprint(outputs_on_step[step]['action_mask'])
    
    


Step: 0, sample all actions under action_mask(bruteforce search)
	{'local_vulnerability': array([0, 2], dtype=int32)}	local_vulnerability(`client, SearchEdgeHistory)
	{'remote_vulnerability': array([0, 0, 7], dtype=int32)}	remote_vulnerability(`client, `client, ScanSharepointParentDirectory)
	{'remote_vulnerability': array([0, 0, 3], dtype=int32)}	remote_vulnerability(`client, `client, NavigateWebDirectory)
	{'remote_vulnerability': array([0, 0, 5], dtype=int32)}	remote_vulnerability(`client, `client, ScanPageContent)
	{'remote_vulnerability': array([0, 0, 6], dtype=int32)}	remote_vulnerability(`client, `client, ScanPageSource)
	{'remote_vulnerability': array([0, 0, 0], dtype=int32)}	remote_vulnerability(`client, `client, AccessDataWithSASToken)
	{'remote_vulnerability': array([0, 0, 1], dtype=int32)}	remote_vulnerability(`client, `client, CredScanGitHistory)
	{'remote_vulnerability': array([0, 0, 2], dtype=int32)}	remote_vulnerability(`client, `client, ListAzureResources)
	{'remote_v

,internal index,status,properties,discovered node properties,local_attacks,remote_attacks,gathered_credentials (no restrict to node)
id,,,,,,,
client,0,owned,[],[],[SearchEdgeHistory],[],{}
Website,1,discovered,"[MySql, Ubuntu, nginx/1.10.3]",[],None,"[ScanPageContent, ScanPageSource]",{}



Step: 1, sample all actions under action_mask(bruteforce search)
	{'local_vulnerability': array([0, 2], dtype=int32)}	local_vulnerability(`client, SearchEdgeHistory)
	{'remote_vulnerability': array([0, 1, 6], dtype=int32)}	remote_vulnerability(`client, `Website, ScanPageSource)
	{'remote_vulnerability': array([0, 1, 3], dtype=int32)}	remote_vulnerability(`client, `Website, NavigateWebDirectory)
	{'remote_vulnerability': array([0, 0, 7], dtype=int32)}	remote_vulnerability(`client, `client, ScanSharepointParentDirectory)
	{'remote_vulnerability': array([0, 1, 4], dtype=int32)}	remote_vulnerability(`client, `Website, NavigateWebDirectoryFurther)
	{'remote_vulnerability': array([0, 1, 5], dtype=int32)}	remote_vulnerability(`client, `Website, ScanPageContent)
	{'remote_vulnerability': array([0, 0, 3], dtype=int32)}	remote_vulnerability(`client, `client, NavigateWebDirectory)
	{'remote_vulnerability': array([0, 1, 2], dtype=int32)}	remote_vulnerability(`client, `Website, ListAzureResources)

,internal index,status,properties,discovered node properties,local_attacks,remote_attacks,gathered_credentials (no restrict to node)
id,,,,,,,
client,0,owned,[],[],[SearchEdgeHistory],[],{}
Website,1,discovered,"[MySql, Ubuntu, nginx/1.10.3]",[],None,"[ScanPageContent, ScanPageSource]",{}
Website.Directory,2,discovered,"[Ubuntu, nginx/1.10.3, CTFFLAG:Readme.txt-Disc...",[],None,"[NavigateWebDirectoryFurther, NavigateWebDirec...",{}



Step: 2, sample all actions under action_mask(bruteforce search)
	{'local_vulnerability': array([0, 2], dtype=int32)}	local_vulnerability(`client, SearchEdgeHistory)
	{'remote_vulnerability': array([0, 2, 2], dtype=int32)}	remote_vulnerability(`client, `Website.Directory, ListAzureResources)
	{'remote_vulnerability': array([0, 1, 1], dtype=int32)}	remote_vulnerability(`client, `Website, CredScanGitHistory)
	{'remote_vulnerability': array([0, 0, 4], dtype=int32)}	remote_vulnerability(`client, `client, NavigateWebDirectoryFurther)
	{'remote_vulnerability': array([0, 0, 1], dtype=int32)}	remote_vulnerability(`client, `client, CredScanGitHistory)
	{'remote_vulnerability': array([0, 0, 0], dtype=int32)}	remote_vulnerability(`client, `client, AccessDataWithSASToken)
	{'remote_vulnerability': array([0, 2, 1], dtype=int32)}	remote_vulnerability(`client, `Website.Directory, CredScanGitHistory)
	{'remote_vulnerability': array([0, 0, 5], dtype=int32)}	remote_vulnerability(`client, `client, ScanP

,internal index,status,properties,discovered node properties,local_attacks,remote_attacks,gathered_credentials (no restrict to node)
id,,,,,,,
client,0,owned,[],[],[SearchEdgeHistory],[],{}
Website,1,discovered,"[MySql, Ubuntu, nginx/1.10.3]",[],None,"[ScanPageContent, ScanPageSource]",{}
Website.Directory,2,discovered,"[Ubuntu, nginx/1.10.3, CTFFLAG:Readme.txt-Disc...",[],None,"[NavigateWebDirectoryFurther, NavigateWebDirec...",{}
GitHubProject,3,discovered,"[GitHub, SasUrlInCommit]",[],None,[CredScanGitHistory],{}



Step: 3, sample all actions under action_mask(bruteforce search)
	{'local_vulnerability': array([0, 2], dtype=int32)}	local_vulnerability(`client, SearchEdgeHistory)
	{'remote_vulnerability': array([0, 3, 0], dtype=int32)}	remote_vulnerability(`client, `GitHubProject, AccessDataWithSASToken)
	{'remote_vulnerability': array([0, 0, 5], dtype=int32)}	remote_vulnerability(`client, `client, ScanPageContent)
	{'remote_vulnerability': array([0, 1, 6], dtype=int32)}	remote_vulnerability(`client, `Website, ScanPageSource)
	{'remote_vulnerability': array([0, 1, 4], dtype=int32)}	remote_vulnerability(`client, `Website, NavigateWebDirectoryFurther)
	{'remote_vulnerability': array([0, 2, 2], dtype=int32)}	remote_vulnerability(`client, `Website.Directory, ListAzureResources)
	{'remote_vulnerability': array([0, 3, 1], dtype=int32)}	remote_vulnerability(`client, `GitHubProject, CredScanGitHistory)
	{'remote_vulnerability': array([0, 1, 5], dtype=int32)}	remote_vulnerability(`client, `Website, ScanPag

,internal index,status,properties,discovered node properties,local_attacks,remote_attacks,gathered_credentials (no restrict to node)
id,,,,,,,
client,0,owned,[],[],[SearchEdgeHistory],[],{}
Website,1,discovered,"[MySql, Ubuntu, nginx/1.10.3]",[],None,"[ScanPageContent, ScanPageSource]",{}
Website.Directory,2,discovered,"[Ubuntu, nginx/1.10.3, CTFFLAG:Readme.txt-Disc...",[],None,"[NavigateWebDirectoryFurther, NavigateWebDirec...",{}
GitHubProject,3,discovered,"[GitHub, SasUrlInCommit]",[],None,[CredScanGitHistory],{}



Step: 4, sample all actions under action_mask(bruteforce search)
	{'local_vulnerability': array([0, 2], dtype=int32)}	local_vulnerability(`client, SearchEdgeHistory)
	{'remote_vulnerability': array([0, 2, 5], dtype=int32)}	remote_vulnerability(`client, `Website.Directory, ScanPageContent)
	{'remote_vulnerability': array([0, 2, 2], dtype=int32)}	remote_vulnerability(`client, `Website.Directory, ListAzureResources)
	{'remote_vulnerability': array([0, 1, 2], dtype=int32)}	remote_vulnerability(`client, `Website, ListAzureResources)
	{'remote_vulnerability': array([0, 3, 2], dtype=int32)}	remote_vulnerability(`client, `GitHubProject, ListAzureResources)
	{'remote_vulnerability': array([0, 0, 2], dtype=int32)}	remote_vulnerability(`client, `client, ListAzureResources)
	{'remote_vulnerability': array([0, 3, 3], dtype=int32)}	remote_vulnerability(`client, `GitHubProject, NavigateWebDirectory)
	{'remote_vulnerability': array([0, 3, 1], dtype=int32)}	remote_vulnerability(`client, `GitHubProject

interactive(children=(IntSlider(value=1, description='step', max=3, min=-1), Output()), _dom_classes=('widget-…

In [136]:
outputs_on_step[2]['action_mask']['connect'].shape

(3, 3, 3, 5)